In [2]:
import re
import time
from typing import Iterator
import requests
import lxml
import lxml.html
import cssselect
from pymongo import MongoClient

import numpy as np
import pandas as pd
import itertools

In [5]:
#　クローリング部分
def main():
    """
    クローラーのメインの処理。
    """
    client = MongoClient('localhost', 27017)  # ローカルホストのMongoDBに接続する。
    collection = client.scraping.coe_auction  # scrapingデータベースのcoe_auctionコレクションを得る。
    collection.create_index('key',unique=False) # データを識別するキーを格納するkeyフィールドにインデックスを作成する。
    
    headers = requests.utils.default_headers()
    headers['User-Agent'] = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36'
    
    #例外処理
    #honduras2018
    url = 'https://allianceforcoffeeexcellence.org/honduras-2018/'
    key = extract_key(url)
    coe = collection.find_one({'key': key})  # MongoDBからkeyに該当するデータを探す。
    if not coe:
        time.sleep(3)
        response_honduras_2018 = requests.get(url)
        tables_honduras_2018 = pd.read_html(response_honduras_2018.content, match='\$\d{1}')
        cols=pd.DataFrame(['Lot #', 'Winning Farm / CWS', 'Lot Size', 'High Bid', 'Total Value', 'High Bidder(s)']).T
        tables_honduras_2018[1]=pd.concat([cols, tables_honduras_2018[1]], join='outer')#先頭に列データを挿入
        tables_honduras_2018[2]=tables_honduras_2018[2].drop(42).drop(tables_honduras_2018[2].columns[[7]], axis=1)#不必要な列と最終行の削除
        results = scrape_detail_page(response_honduras_2018, tables_honduras_2018)
        collection.insert_many(results)
        print(len(results), url, sep='\n')
    #nicaragua2018
    url = 'https://allianceforcoffeeexcellence.org/nicaragua-2018/'
    key = extract_key(url)
    coe = collection.find_one({'key': key})  # MongoDBからkeyに該当するデータを探す。
    if not coe:
        time.sleep(3)
        response_nicaragua_2018 = requests.get(url)
        tables_nicaragua_2018 = pd.read_html(response_nicaragua_2018.content, match='[tT].... [vV]....')
        cols,data,df=[],[],[]#multi-index形式となった全データを展開しDataFrame化
        for i in range(len(tables_nicaragua_2018[0].columns)):
            cols.append(tables_nicaragua_2018[0].columns[i][0])
            for j in range(len(tables_nicaragua_2018[0].columns[i])-1):
                data.append(tables_nicaragua_2018[0].columns[i][j+1])
        for i in range(0, len(data), 3):
            df.append(data[i: i+3])
        tables_nicaragua_2018[0]=pd.DataFrame(df).T.rename(columns=pd.Series(cols)).drop(columns=['Size', 'Lot #'])
        results = scrape_detail_page(response_nicaragua_2018, tables_nicaragua_2018)
        collection.insert_many(results)
        print(len(results), url, sep='\n')
    #bolivia2004
    url = 'https://allianceforcoffeeexcellence.org/bolivia-2004/'
    key = extract_key(url)
    coe = collection.find_one({'key': key})  # MongoDBからkeyに該当するデータを探す。
    if not coe:
        time.sleep(3)
        response_bolivia_2004 = requests.get(url)#別年度のデータが混入
        tables_bolivia_2004 = pd.read_html(response_bolivia_2004.content, match='[tT].... [vV]....')
        tables_bolivia_2004.pop(1)
        results = scrape_detail_page(response_bolivia_2004, tables_bolivia_2004)
        collection.insert_many(results)
        print(len(results), url, sep='\n')
    #mexico2018
    url = 'https://allianceforcoffeeexcellence.org/mexico-2018/'
    key = extract_key(url)
    coe = collection.find_one({'key': key})  # MongoDBからkeyに該当するデータを探す。
    if not coe:
        time.sleep(3)
        response_mexico_2018 = requests.get(url)#分割されてしまったNWデータを統合
        tables_mexico_2018 = pd.read_html(response_mexico_2018.content, match='[tT][oO][tT].. [vV]....')
        tables_mexico_2018[0].iloc[2][3]=tables_mexico_2018[0].iloc[2][3] + ' ' + tables_mexico_2018[0].iloc[3][3]
        tables_mexico_2018[0].iloc[4][3]=tables_mexico_2018[0].iloc[4][3] + ', ' + tables_mexico_2018[0].iloc[5][3]
        tables_mexico_2018[0]=tables_mexico_2018[0].drop(index=tables_mexico_2018[0].index[[3, 5]])
        results = scrape_detail_page(response_mexico_2018, tables_mexico_2018)
        collection.insert_many(results)
        print(len(results), url, sep='\n')

    #main
    session = requests.Session()
    response = session.get('https://allianceforcoffeeexcellence.org/competition-auction-results/')  # 一覧ページを取得する。
    urls = scrape_list_page(response)  # scrape関数を用いて詳細ページのURL(~/country-year/~)一覧を得る。

    for url in urls:
        key = extract_key(url)  # URLからキー(country-year)を取得する。
        coe = collection.find_one({'key': key})  # MongoDBからkeyに該当するデータを探す。
        print(url)
        if not coe:  # MongoDBに存在しない場合だけ、詳細ページをクロールする。
            time.sleep(3)
            response = session.get(url)  # 詳細ページを取得する。
            try:
                tables = pd.read_html(response.content, match='[tT][oO][tT][aA][lL] [vV][aA][lL][uU][eE]')#Total Value条件で絞る
                results = scrape_detail_page(response, tables)
                collection.insert_many(results)  # 表の情報をMongoDBのcoe_auctionコレクションに保存する。
                print(len(results), url, sep='\n')
            except TypeError:
                continue
            except ValueError:
                continue

#　スクレイピング部分
def scrape_list_page(response: requests.Response) -> Iterator[str]:
    """
    一覧ページのResponseから詳細ページのURLを抜き出すジェネレーター関数。
    """
    html = lxml.html.fromstring(response.text)
    html.make_links_absolute(response.url)
    urls = []
    for a in html.cssselect('#menu-coe-country-programs-menu a[href^="https://"]'):
        urls.append(a.get('href'))
    urls = set(urls)
    urls.remove('https://allianceforcoffeeexcellence.org/competition-auction-results/')
    urls.remove('https://allianceforcoffeeexcellence.org/farm-directory/')
    #例外処理
    urls.remove('https://allianceforcoffeeexcellence.org/nicaragua-2002/')
    return urls

#  結果のスクレイピング用
def scrape_detail_page(response: requests.Response, tables: list) -> dict:
    
    #COE Auction ResultsとOrganizing Country Commissionの結合
    for x in itertools.combinations(tables, 2):  #  tablesの内、要素数が等しいtableを抽出
        if abs(len(x[0]) - len(x[1]))<=2:
            left, right=x[0], x[1]
            #columnsが数字なら1行目をcolumnsに
            left=fix_columns(delete_last_row(left))
            right=fix_columns(delete_last_row(right))
            df=pd.concat([left, right], axis=1, join='outer')
            #重複列の削除
            df=df.loc[:,~df.columns.duplicated()]
            break
            
    #NW Auction Resultsを検索しjoin
    if len(tables)==3:#NWが存在時、table数は3
        l=list(map(len, tables))
        NW=tables[l.index(min(l))]#最も要素数の少ないテーブルがNW
        NW=fix_columns(NW)
        NW=delete_last_row(NW)
        #全データの結合
        df=pd.concat([df, NW], join='outer')
        
    #COEのみ
    elif len(tables)==1:
        df=fix_columns(tables[0])
        df=delete_last_row(df)
        
    #全て欠損値の行を削除
    df=df.dropna(how='all')
    #無駄な列の削除
#     df=df.drop(['Region', 'Country', 'Variety Processing', 'Variety', 'Processing', 'Process, Variety', 'Process'], axis=1)
    #keyの追加
    df['key'] = extract_key(response.url)  
    #dict形式に変更
    df=df.to_dict(orient='record')
    return df

def extract_key(url: str) -> str:
    """
    URLからキー（URLの末尾のISBN）を抜き出す。
    """
    m = re.search(r'/([^/]+)/$', url)  # 　　"/country-year/"を正規表現で取得。
    return m.group(1)

def fix_columns(df):
    df.reset_index(inplace=True, drop=True)#index番号の修正
    if df.columns.dtype=='int64':#columnsがなければ（整数型だったら）1行目をcolumnnに
        df.rename(columns=df.iloc[0], inplace=True)#columnsにはseriesを渡す
        df.drop(df.index[0], inplace=True)#1行目の削除
        df.reset_index(inplace=True, drop=True)#index番号の修正
    #列名修正
    df=df.rename(columns=str.title).rename(columns=lambda s: s.replace('/', ' ').replace('  ', ''))
    df.rename(columns={
        'Auction Commission': 'Commissions', 'Comission':'Commissions', 'Commission':'Commissions', 'Comissions':'Commissions',
        'Winning Farm Cws':'Farm Cws', 
        'Nome': 'Name'
        }, inplace=True)
    return df
    
def delete_last_row(df):
    last_row=df.iloc[-1,:].values.tolist()
    if ('Stats:' in last_row) or ('Stats' in last_row) or ('Totals:' in last_row) or ('Totals' in last_row) or ('Total:' in last_row) or ('Total' in last_row):#最終行の先頭にStat/Totals/nanがあるか
        df=df.drop(df.index[-1])
    elif pd.isna(last_row[0]):#最終行の先頭にnanがあるか
        df=df.drop(df.index[-1])
    return df

if __name__ == '__main__':
    main()

https://allianceforcoffeeexcellence.org/brazil-pulped-naturals-2017/
https://allianceforcoffeeexcellence.org/colombia-north-2009/
https://allianceforcoffeeexcellence.org/costa-rica-2019/
https://allianceforcoffeeexcellence.org/mexico-2015/
https://allianceforcoffeeexcellence.org/peru-2021/
https://allianceforcoffeeexcellence.org/rwanda-2012/
https://allianceforcoffeeexcellence.org/brazil-naturals-2014/
23
https://allianceforcoffeeexcellence.org/brazil-naturals-2014/
https://allianceforcoffeeexcellence.org/brazil-pulped-naturals-1999/
https://allianceforcoffeeexcellence.org/brazil-2019/
https://allianceforcoffeeexcellence.org/el-salvador-2008/
https://allianceforcoffeeexcellence.org/brazil-pulped-naturals-2005/
https://allianceforcoffeeexcellence.org/colombia-south-2005/
https://allianceforcoffeeexcellence.org/rwanda-2013/
https://allianceforcoffeeexcellence.org/ecuador-2021/
https://allianceforcoffeeexcellence.org/el-salvador-2015/
https://allianceforcoffeeexcellence.org/honduras-2017/

/var/folders/k4/qxx51vq90l957g_z0g8nnqyc0000gn/T/ipykernel_25281/1161707121.py:149: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  df=df.to_dict(orient='record')
